# Use Public Resources: Query keys

[![Open in Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/jburel/EOSC-14176/blob/main/notebooks/query_keys.ipynb)

## Learning Objectives

* How to access data stored in [Preclinical image dataset repository](https://pidar.hpc4ai.unito.it/)(PIDAR)
* How to access data stored in [Image Data Resource](https://idr.openmicroscopy.org/) (IDR)

In [22]:
import requests
import json

## First retrieve information about datasets stored in PIDAR
We load and parse the ``json`` file.

In [23]:
UNITO_URL_JSON = "https://pidar.hpc4ai.unito.it/file.json"
import urllib.request
with urllib.request.urlopen(UNITO_URL_JSON) as url:
    data = json.load(url)

### Find values associated to a key
In the example below, we look at the ``Cell line``.

In [24]:
KEY = "Cell line"

In [25]:
values = []
for d in data:
    value = d[KEY]
    if value is not None and value != "":
        values.append(value)
pidar_values = list(set(values))
print(pidar_values)

['Vk12598 cells (cell line 1); JJN3 HMCL cells (cell line 2)', 'U87.CXCR4', 'B16-F10', 'pH8N8', 'PC3-PIP and PC3-FLU', 'P3, patient derived']


## Find information in IDR

In [11]:
INDEX_PAGE = "https://idr.openmicroscopy.org/webclient/?experimenter=-1"

# create http session
with requests.Session() as session:
    request = requests.Request('GET', INDEX_PAGE)
    prepped = session.prepare_request(request)
    response = session.send(prepped)
    if response.status_code != 200:
        response.raise_for_status()

In [12]:
BASE_URL = "https://idr.openmicroscopy.org/searchengine/api/v1/resources/{type}/"
KEYS_SEARCH = BASE_URL + "searchvaluesusingkey/?key={key}"
KEY_VALUE_SEARCH = BASE_URL + "search/?key={key}&value={value}"

In [13]:
%%time
qs1 = {'type': 'image', 'key': KEY.lower()}
url = KEYS_SEARCH.format(**qs1)
json = session.get(url).json()

CPU times: user 75.4 ms, sys: 6.34 ms, total: 81.7 ms
Wall time: 360 ms


### Find all the values associated to a given key in IDR

In [16]:
data = json['data']
idr_values = []
for k in data:
    idr_values.append(k['Value'])

### Compare list of results

In [17]:
common_values = [d for d in idr_values if d in pidar_values]

### Retrieve the image ID and the associated metadata for a given value in IDR

In [37]:
value = idr_values[0]
qs1 = {'type': 'image', 'key': KEY.lower(), 'value': value}
url = KEY_VALUE_SEARCH.format(**qs1)
json_result = session.get(url).json()
images = json_result['results']['results']
print(images)

[{'dataset_id': None, 'dataset_name': None, 'description': None, 'doc_type': 'image_keyvalue_pair_metadata', 'experiment': None, 'group_id': 3, 'id': 2868078, 'key_values': [{'index': 0.0, 'name': 'Cell Line', 'value': 'LM2'}, {'index': 0.0, 'name': 'Characteristics Comments', 'value': 'LM2 is derived from MDA-MB-231'}, {'index': 0.0, 'name': 'Gene Identifier', 'value': '11214'}, {'index': 0.0, 'name': 'Organism', 'value': 'Homo sapiens'}, {'index': 0.0, 'name': 'Phenotype', 'value': 'Low total YAP/TAZ in nuclear and ring regions normalised to nuclear area'}, {'index': 0.0, 'name': 'Quality Control', 'value': 'pass'}, {'index': 0.0, 'name': 'RefSeq Locus', 'value': 'NM_006738'}, {'index': 0.0, 'name': 'siRNA Library Name', 'value': 'Dharmacon siGENOME'}, {'index': 0.0, 'name': 'siRNA Pool Identifier', 'value': 'M-008868-01'}, {'index': 1.0, 'name': 'Channels', 'value': 'AlexaFluor647:YAP/TAZ; AlexaFluor568:alphaTubulin;Phalloidin488: F-actin;Hoechst: nuclei'}, {'index': 1.0, 'name': 'G

### License (BSD 2-Clause)¶

Copyright (C) 2023 University of Dundee. All Rights Reserved.

Redistribution and use in source and binary forms, with or without modification, are permitted provided that the following conditions are met:

Redistributions of source code must retain the above copyright notice, this list of conditions and the following disclaimer. Redistributions in binary form must reproduce the above copyright notice, this list of conditions and the following disclaimer in the documentation and/or other materials provided with the distribution. THIS SOFTWARE IS PROVIDED BY THE COPYRIGHT HOLDERS AND CONTRIBUTORS "AS IS" AND ANY EXPRESS OR IMPLIED WARRANTIES, INCLUDING, BUT NOT LIMITED TO, THE IMPLIED WARRANTIES OF MERCHANTABILITY AND FITNESS FOR A PARTICULAR PURPOSE ARE DISCLAIMED. IN NO EVENT SHALL THE COPYRIGHT OWNER OR CONTRIBUTORS BE LIABLE FOR ANY DIRECT, INDIRECT, INCIDENTAL, SPECIAL, EXEMPLARY, OR CONSEQUENTIAL DAMAGES (INCLUDING, BUT NOT LIMITED TO, PROCUREMENT OF SUBSTITUTE GOODS OR SERVICES; LOSS OF USE, DATA, OR PROFITS; OR BUSINESS INTERRUPTION) HOWEVER CAUSED AND ON ANY THEORY OF LIABILITY, WHETHER IN CONTRACT, STRICT LIABILITY, OR TORT (INCLUDING NEGLIGENCE OR OTHERWISE) ARISING IN ANY WAY OUT OF THE USE OF THIS SOFTWARE, EVEN IF ADVISED OF THE POSSIBILITY OF SUCH DAMAGE. 